<a href="https://colab.research.google.com/github/optimalMachine/UpbitCryptocurrencyClusteringAnalysis/blob/main/Upbit_Cryptocurrency_Clustering_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
import requests
from datetime import datetime, timedelta
import time
import warnings
import logging

warnings.filterwarnings('ignore')

# 로깅 설정 - 기존 설정과 충돌 방지
logger = logging.getLogger(__name__)
if not logger.handlers:
    handler = logging.StreamHandler()
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    logger.setLevel(logging.INFO)

# 영어 폰트 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

def validate_data_continuity(df, market, expected_interval_hours=4):
    """데이터 연속성 검증 및 보고"""
    if len(df) < 2:
        return True

    time_diffs = df.index.to_series().diff()[1:]
    expected_diff = pd.Timedelta(hours=expected_interval_hours)

    # 허용 오차 (5%)
    tolerance = 0.05
    gaps = time_diffs[
        (time_diffs < expected_diff * (1 - tolerance)) |
        (time_diffs > expected_diff * (1 + tolerance))
    ]

    if len(gaps) > 0:
        logger.warning(f"{market}: 비정상적인 시간 간격 {len(gaps)}개 발견")
        # 큰 갭만 보고
        large_gaps = gaps[gaps > expected_diff * 2]
        if len(large_gaps) > 0:
            logger.warning(f"{market}: 큰 데이터 갭 {len(large_gaps)}개 (2배 이상 간격)")

    return len(gaps) == 0

def get_upbit_candles(market, hours=4320, interval='240'):
    """
    업비트에서 특정 코인의 과거 가격 데이터를 가져오는 함수 - 개선된 버전

    Parameters:
    - market: 마켓 코드 (예: 'KRW-BTC')
    - hours: 가져올 데이터 기간 (시간 단위, 기본값: 4320시간 = 180일 = 6개월)
    - interval: 캔들 간격 ('1', '3', '5', '15', '10', '30', '60', '240')
    """

    # 4시간봉 = 240분
    url = f"https://api.upbit.com/v1/candles/minutes/{interval}"

    # API는 최대 200개까지만 반환하므로 여러 번 호출 필요
    all_data = []
    to_time = None
    collected_times = set()  # 전체 수집된 시간을 추적

    # 4시간봉 기준으로 계산
    total_candles_needed = hours // 4

    headers = {"accept": "application/json"}

    # API 호출 횟수 제한 (최대 10회 = 2000개 캔들)
    max_api_calls = 10
    api_calls = 0

    while len(all_data) < total_candles_needed and api_calls < max_api_calls:
        params = {
            'market': market,
            'count': min(200, total_candles_needed - len(all_data))
        }

        if to_time:
            params['to'] = to_time

        try:
            response = requests.get(url, params=params, headers=headers)
            if response.status_code == 200:
                data = response.json()
                if not data:
                    break

                # 중복 제거 로직 (개선됨)
                new_data = []
                for candle in data:
                    candle_time = candle['candle_date_time_kst']
                    if candle_time not in collected_times:
                        new_data.append(candle)
                        collected_times.add(candle_time)

                if not new_data:
                    logger.warning(f"{market}: 새로운 데이터가 없습니다.")
                    break

                # API 응답 데이터 정렬 확인
                if len(new_data) > 1:
                    first_time = pd.to_datetime(new_data[0]['candle_date_time_kst'])
                    last_time = pd.to_datetime(new_data[-1]['candle_date_time_kst'])

                    # 업비트 API는 보통 최신 데이터부터 반환 (내림차순)
                    if first_time > last_time:  # 내림차순 (최신 → 과거)
                        to_time = new_data[-1]['candle_date_time_utc']
                    else:  # 오름차순 (과거 → 최신) - 드물지만 가능
                        to_time = new_data[0]['candle_date_time_utc']

                all_data.extend(new_data)

                # API 호출 제한을 위한 대기
                time.sleep(0.1)
                api_calls += 1

            else:
                logger.warning(f"API 오류 ({market}): {response.status_code}")
                break

        except Exception as e:
            logger.error(f"Error for {market}: {str(e)}")
            break

    if all_data:
        df = pd.DataFrame(all_data)
        df['candle_date_time_kst'] = pd.to_datetime(df['candle_date_time_kst'])
        df = df.set_index('candle_date_time_kst')
        df = df.sort_index(ascending=True)

        # 중복 인덱스 제거 (더 엄격하게)
        df = df[~df.index.duplicated(keep='first')]

        # 데이터 연속성 체크
        validate_data_continuity(df, market)

        # 요청한 기간보다 적게 수집된 경우 알림
        if len(df) < total_candles_needed * 0.8:  # 80% 미만
            print(f"  {market}: {len(df)}개 캔들만 수집됨 (목표: {total_candles_needed}개)")

        return df
    else:
        return None

def calculate_features(returns_df):
    """각 코인의 특성 계산 - 개선된 버전"""
    features = pd.DataFrame()

    for coin in returns_df.columns:
        coin_returns = returns_df[coin].dropna()

        # 최소 데이터 체크
        if len(coin_returns) < 60:  # 10일 최소
            logger.warning(f"{coin}: 데이터 부족 ({len(coin_returns)}개)")
            continue

        try:
            # 기본 통계
            features.loc[coin, 'mean_return'] = coin_returns.mean()
            features.loc[coin, 'volatility'] = coin_returns.std()

            # Sharpe Ratio 계산
            if coin_returns.std() > 1e-8:
                features.loc[coin, 'sharpe_ratio'] = (
                    coin_returns.mean() / coin_returns.std() * np.sqrt(365 * 6)
                )
            else:
                features.loc[coin, 'sharpe_ratio'] = 0

            # 기타 통계량
            features.loc[coin, 'skewness'] = coin_returns.skew()
            features.loc[coin, 'kurtosis'] = coin_returns.kurtosis()
            features.loc[coin, 'max_return'] = coin_returns.max()
            features.loc[coin, 'min_return'] = coin_returns.min()
            features.loc[coin, 'positive_periods_ratio'] = (coin_returns > 0).sum() / len(coin_returns)

            # Max Drawdown - 안전한 계산
            try:
                cumulative = (1 + coin_returns).cumprod()
                running_max = cumulative.expanding().max()
                drawdown = (cumulative - running_max) / running_max
                features.loc[coin, 'max_drawdown'] = drawdown.min()
            except Exception as e:
                logger.warning(f"{coin}: Max Drawdown 계산 실패")
                features.loc[coin, 'max_drawdown'] = np.nan

            # Calmar Ratio
            if abs(features.loc[coin, 'max_drawdown']) > 1e-8:
                features.loc[coin, 'calmar_ratio'] = (
                    features.loc[coin, 'mean_return'] * 365 * 6 / abs(features.loc[coin, 'max_drawdown'])
                )
            else:
                features.loc[coin, 'calmar_ratio'] = 0

            # 동적 기간 계산 (데이터 길이에 따라)
            data_len = len(coin_returns)

            # 일중 변동성 (최소 6개 필요)
            if data_len >= 6:
                daily_returns = coin_returns.rolling(6).sum().dropna()
                if len(daily_returns) > 0:
                    features.loc[coin, 'intraday_volatility'] = daily_returns.std()

            # 주간 변동성 (최소 42개 필요)
            if data_len >= 42:
                weekly_returns = coin_returns.rolling(42).sum().dropna()
                if len(weekly_returns) > 0:
                    features.loc[coin, 'weekly_volatility'] = weekly_returns.std()

            # 월간 트렌드 (최소 180개 필요)
            if data_len >= 180:
                monthly_return = coin_returns.iloc[-180:].sum()
                features.loc[coin, 'monthly_trend'] = monthly_return

            # 세션별 성과 (수정된 안전한 버전)
            if hasattr(returns_df.index, 'hour') and data_len >= 30:
                try:
                    # returns_df에서 해당 코인의 데이터만 추출
                    coin_series = returns_df[coin].dropna()

                    if len(coin_series) > 0:
                        coin_hours = coin_series.index.hour

                        # 각 세션의 데이터 필터링
                        asia_data = coin_series[coin_hours.isin([0, 4])]
                        europe_data = coin_series[coin_hours.isin([8, 12])]
                        us_data = coin_series[coin_hours.isin([16, 20])]

                        # 세션별 수익률 계산
                        if len(asia_data) > 10:
                            features.loc[coin, 'asia_session_return'] = asia_data.mean()
                        if len(europe_data) > 10:
                            features.loc[coin, 'europe_session_return'] = europe_data.mean()
                        if len(us_data) > 10:
                            features.loc[coin, 'us_session_return'] = us_data.mean()
                except Exception as e:
                    logger.debug(f"{coin}: 세션별 성과 계산 실패 - {str(e)}")

        except Exception as e:
            logger.error(f"{coin} 특성 계산 실패: {str(e)}")
            continue

    # NaN 값 체크 및 제거
    if features.isnull().any().any():
        features_before = len(features)
        # 핵심 컬럼에 NaN이 있는 행만 제거
        essential_cols = ['mean_return', 'volatility', 'sharpe_ratio']
        features = features.dropna(subset=essential_cols)
        if features_before > len(features):
            print(f"⚠️ 필수 특성에 NaN 값이 있는 {features_before - len(features)}개 코인 제외")

    return features

def classify_coins_by_data_quality(coin_data_info):
    """데이터 품질에 따른 코인 분류"""
    high_quality = []    # 90% 이상 (972개+)
    medium_quality = []  # 50-90% (540-972개)
    low_quality = []     # 50% 미만 (540개 미만)

    target_candles = 1080  # 6개월 목표

    for coin, count in coin_data_info.items():
        if count >= target_candles * 0.9:
            high_quality.append(coin)
        elif count >= target_candles * 0.5:
            medium_quality.append(coin)
        else:
            low_quality.append(coin)

    return {
        'high': high_quality,
        'medium': medium_quality,
        'low': low_quality
    }

def get_all_krw_markets():
    """업비트의 모든 KRW 마켓 코인 목록을 가져오는 함수"""
    url = "https://api.upbit.com/v1/market/all"
    headers = {"accept": "application/json"}

    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            markets = response.json()
            krw_coins = []

            for market in markets:
                if market['market'].startswith('KRW-'):
                    coin_symbol = market['market'].split('-')[1]
                    krw_coins.append(coin_symbol)

            return krw_coins
        else:
            return []
    except Exception as e:
        logger.error(f"마켓 목록 가져오기 실패: {str(e)}")
        return []

def find_common_index_efficient(returns_data):
    """메모리 효율적인 공통 인덱스 찾기"""
    if not returns_data:
        return pd.DatetimeIndex([])

    # 첫 번째 코인의 인덱스를 기준으로
    common_dates = None
    min_required = 60  # 최소 요구 데이터

    for i, (coin, returns) in enumerate(returns_data.items()):
        if common_dates is None:
            common_dates = set(returns.index)
        else:
            # 교집합 업데이트
            common_dates &= set(returns.index)

        # 공통 날짜가 너무 적으면 조기 종료
        if len(common_dates) < min_required:
            logger.warning(f"공통 날짜가 너무 적습니다: {len(common_dates)}개 (처리된 코인: {i+1}개)")
            break

    if common_dates:
        return pd.DatetimeIndex(sorted(list(common_dates)))
    else:
        return pd.DatetimeIndex([])

def get_multiple_coins_data(coin_list, hours=4320, max_coins=None):
    """
    여러 코인의 데이터를 한번에 가져오는 함수 - 개선된 버전
    """
    returns_data = {}
    price_data = {}
    failed_coins = []
    coin_data_info = {}  # 각 코인의 데이터 개수 저장

    # 최대 코인 수 제한 (옵션)
    if max_coins:
        coin_list = coin_list[:max_coins]

    total_coins = len(coin_list)
    print(f"총 {total_coins}개 코인의 4시간봉 데이터 수집 시작...")
    print(f"수집 기간: 약 {hours//24}일 ({hours}시간) = {hours//24//30:.1f}개월")
    print(f"예상 캔들 수: 약 {hours//4}개")
    print("=" * 60)

    # 배치 처리를 위한 설정
    batch_size = 20 if total_coins > 100 else min(total_coins, 30)

    for batch_start in range(0, total_coins, batch_size):
        batch_end = min(batch_start + batch_size, total_coins)
        batch_coins = coin_list[batch_start:batch_end]

        print(f"\n배치 {batch_start//batch_size + 1}/{(total_coins-1)//batch_size + 1} 처리 중...")

        for idx, coin in enumerate(batch_coins):
            global_idx = batch_start + idx
            market = f'KRW-{coin}'

            # 진행상황 표시
            if (global_idx + 1) % 5 == 0 or global_idx == 0:
                print(f"진행률: {global_idx+1}/{total_coins} ({(global_idx+1)/total_coins*100:.1f}%)")

            try:
                df = get_upbit_candles(market, hours=hours, interval='240')
                if df is not None and len(df) > 60:  # 최소 60개 캔들 (10일) 이상
                    # 데이터 정렬 확인
                    df = df.sort_index(ascending=True)

                    # 수익률 계산
                    returns = df['trade_price'].pct_change().dropna()

                    if len(returns) > 60 and returns.std() > 1e-8:
                        returns_data[coin] = returns
                        price_data[coin] = df['trade_price']
                        coin_data_info[coin] = len(returns)
                    else:
                        failed_coins.append(coin)
                else:
                    failed_coins.append(coin)
            except Exception as e:
                logger.error(f"{coin} 수집 실패: {str(e)}")
                failed_coins.append(coin)

            # API 호출 제한을 위한 대기
            time.sleep(0.3)

        # 배치 간 추가 대기
        if batch_end < total_coins:
            print(f"다음 배치 처리를 위해 잠시 대기 중...")
            time.sleep(3)

    print(f"\n✅ 데이터 수집 완료!")
    print(f"성공: {len(returns_data)}개 / 실패: {len(failed_coins)}개")

    if failed_coins and len(failed_coins) <= 20:
        print(f"실패한 코인: {', '.join(failed_coins)}")
    elif failed_coins:
        print(f"실패한 코인: {', '.join(failed_coins[:20])} 외 {len(failed_coins)-20}개")

    # 데이터 품질 분류
    quality_groups = classify_coins_by_data_quality(coin_data_info)
    print(f"\n📊 데이터 품질 분석:")
    print(f"  - 고품질 (90%+): {len(quality_groups['high'])}개")
    print(f"  - 중품질 (50-90%): {len(quality_groups['medium'])}개")
    print(f"  - 저품질 (<50%): {len(quality_groups['low'])}개")

    # DataFrame 생성 - 메모리 효율적인 공통 인덱스 찾기
    if returns_data:
        # 효율적인 공통 인덱스 계산
        common_index = find_common_index_efficient(returns_data)

        print(f"\n공통 인덱스 개수: {len(common_index)}")

        # 공통 인덱스로 DataFrame 생성
        returns_dict_aligned = {}
        price_dict_aligned = {}

        # 최소 360개 이상의 공통 데이터 선호
        min_required = 360 if len(common_index) >= 360 else 60

        if len(common_index) >= min_required:
            for coin, returns in returns_data.items():
                try:
                    returns_dict_aligned[coin] = returns[common_index]
                    if coin in price_data:
                        price_dict_aligned[coin] = price_data[coin][common_index]
                except Exception as e:
                    logger.warning(f"{coin}: 공통 인덱스 적용 실패 - {str(e)}")
        else:
            print(f"⚠️ 공통 인덱스 데이터가 부족합니다: {len(common_index)}개")
            # 각 코인별로 가능한 데이터 사용
            for coin, returns in returns_data.items():
                if len(returns) >= min_required:
                    returns_dict_aligned[coin] = returns
                    if coin in price_data:
                        price_dict_aligned[coin] = price_data[coin]

        returns_df = pd.DataFrame(returns_dict_aligned)
        price_df = pd.DataFrame(price_dict_aligned)

        # 인덱스 정렬 및 데이터 정렬
        returns_df = returns_df.sort_index(ascending=True)
        price_df = price_df.sort_index(ascending=True)

        # 컬럼도 알파벳 순으로 정렬 (재현성을 위해)
        returns_df = returns_df.reindex(sorted(returns_df.columns), axis=1)
        price_df = price_df.reindex(sorted(price_df.columns), axis=1)

        print(f"\n수집된 공통 4시간봉 개수: {len(returns_df)}개")
        if len(returns_df) > 0:
            print(f"공통 수집 기간: {returns_df.index[0].strftime('%Y-%m-%d %H:%M')} ~ {returns_df.index[-1].strftime('%Y-%m-%d %H:%M')}")
            actual_days = (returns_df.index[-1] - returns_df.index[0]).days
            print(f"실제 수집 일수: {actual_days}일 ({actual_days/30:.1f}개월)")
    else:
        returns_df = pd.DataFrame()
        price_df = pd.DataFrame()

    return returns_df, price_df, quality_groups

def adaptive_clustering(features_df, max_k=10):
    """데이터 특성에 맞는 최적 클러스터 수 찾기 - 수정된 버전"""
    n_samples = len(features_df)

    # 샘플 수에 따라 최대 클러스터 조정
    max_k = min(max_k, n_samples // 10)
    max_k = max(max_k, 3)

    # DataFrame이면 컬럼 체크 가능
    if isinstance(features_df, pd.DataFrame) and 'volatility' in features_df.columns:
        volatility_std = features_df['volatility'].std()
        if volatility_std > 0.02:  # 변동성 차이가 크면
            max_k = min(max_k + 2, n_samples // 8)

    return max_k

def find_optimal_clusters(features_scaled, features_df=None, max_k=10):
    """최적의 클러스터 수 찾기 - 개선된 버전"""
    n_samples = len(features_scaled)

    if features_df is not None:
        max_k = adaptive_clustering(features_df, max_k=max_k)
    else:
        max_k = min(max_k, n_samples // 10)
        max_k = max(max_k, 3)

    inertias = []
    silhouette_scores = []

    for k in range(2, max_k+1):
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans.fit(features_scaled)
        inertias.append(kmeans.inertia_)

        # 안전한 silhouette score 계산
        try:
            score = silhouette_score(features_scaled, kmeans.labels_)
            silhouette_scores.append(score)
        except Exception as e:
            logger.warning(f"Silhouette score 계산 실패 (k={k}): {str(e)}")
            silhouette_scores.append(-1)

    return inertias, silhouette_scores

def perform_clustering_analysis(returns_df):
    """클러스터링 분석 수행 - 개선된 버전"""

    # 데이터 검증
    if returns_df.empty or len(returns_df.columns) < 3:
        print("❌ 분석하기에 충분한 데이터가 없습니다. (최소 3개 이상의 코인 필요)")
        return None, None

    # 특성 계산
    features = calculate_features(returns_df)

    # features가 비어있는지 확인
    if features.empty:
        print("❌ 유효한 특성을 계산할 수 없습니다.")
        return None, None

    # features와 returns_df의 인덱스 일치 확인
    common_coins = list(set(returns_df.columns) & set(features.index))
    if len(common_coins) < 3:
        print("❌ 유효한 특성을 가진 코인이 부족합니다.")
        return None, None

    # 공통 코인만 사용 (정렬하여 재현성 확보)
    common_coins = sorted(common_coins)
    features = features.loc[common_coins]
    returns_df = returns_df[common_coins]

    # 특성 정규화
    scaler = StandardScaler()
    try:
        features_scaled = scaler.fit_transform(features)
    except Exception as e:
        print(f"❌ 특성 정규화 실패: {str(e)}")
        return None, None

    # 최적 클러스터 수 찾기 - features DataFrame 전달
    inertias, silhouette_scores = find_optimal_clusters(features_scaled, features_df=features, max_k=8)

    # 시각화를 위한 Figure 생성
    fig = plt.figure(figsize=(20, 15))

    # 1. Elbow Method
    ax1 = plt.subplot(3, 3, 1)
    ax1.plot(range(2, len(inertias)+2), inertias, 'bo-')
    ax1.set_xlabel('Number of Clusters')
    ax1.set_ylabel('Inertia')
    ax1.set_title('Elbow Method')
    ax1.grid(True, alpha=0.3)

    # 2. Silhouette Score
    ax2 = plt.subplot(3, 3, 2)
    valid_scores = [s for s in silhouette_scores if s != -1]
    if valid_scores:
        ax2.plot(range(2, len(silhouette_scores)+2), silhouette_scores, 'ro-')
        ax2.set_xlabel('Number of Clusters')
        ax2.set_ylabel('Silhouette Score')
        ax2.set_title('Silhouette Score by Number of Clusters')
        ax2.grid(True, alpha=0.3)

        # 최적 클러스터 수 결정 (수정된 로직)
        if valid_scores:
            # silhouette_scores에서 최대값(valid_scores의 최대값)의 인덱스 찾기
            max_score = max(valid_scores)
            optimal_k = silhouette_scores.index(max_score) + 2
        else:
            optimal_k = 3
    else:
        optimal_k = 3  # 기본값
        ax2.text(0.5, 0.5, 'Silhouette Score calculation failed', ha='center', va='center')

    print(f"\n📊 클러스터링 분석 결과:")
    print("=" * 60)
    if valid_scores:
        print(f"최적 클러스터 수: {optimal_k}개 (Silhouette Score: {max(valid_scores):.3f})")
    else:
        print(f"최적 클러스터 수: {optimal_k}개 (Silhouette Score: N/A)")

    # K-means 클러스터링 수행
    try:
        kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
        clusters = kmeans.fit_predict(features_scaled)
    except Exception as e:
        print(f"❌ 클러스터링 실패: {str(e)}")
        plt.close(fig)
        return None, None

    # 클러스터 결과 저장
    cluster_df = pd.DataFrame({
        'Coin': features.index,
        'Cluster': clusters
    })

    # PCA로 차원 축소 (시각화용) - 수정된 버전
    try:
        # 최소 2개 이상의 특성이 있을 때만 PCA 수행
        if features.shape[1] >= 2:
            n_components = min(2, features.shape[1])
            pca = PCA(n_components=n_components)
            features_pca = pca.fit_transform(features_scaled)
            pca_success = True
        else:
            # 특성이 1개인 경우 처리
            pca_success = False
            features_pca = np.column_stack([features_scaled[:, 0], np.zeros(len(features_scaled))])
            n_components = 1
    except Exception as e:
        logger.warning(f"PCA 변환 경고: {str(e)}")
        pca_success = False
        if features_scaled.shape[1] >= 2:
            features_pca = features_scaled[:, :2]
        else:
            features_pca = np.column_stack([features_scaled[:, 0], np.zeros(len(features_scaled))])
        n_components = 1

    # 3. 클러스터 시각화 (PCA) - 수정된 버전
    ax3 = plt.subplot(3, 3, 3)
    if features_pca.shape[1] >= 2:
        scatter = ax3.scatter(features_pca[:, 0], features_pca[:, 1],
                             c=clusters, cmap='viridis', s=200, alpha=0.6)

        # 코인 이름 표시 (많은 경우 상위 코인만)
        if len(features.index) > 30:
            # 샤프비율 기준 상위 30개
            if 'sharpe_ratio' in features.columns:
                top_coins = features.nlargest(30, 'sharpe_ratio').index
            else:
                top_coins = features.index[:30]

            for i, coin in enumerate(features.index):
                if coin in top_coins:
                    ax3.annotate(coin, (features_pca[i, 0], features_pca[i, 1]),
                                fontsize=8, ha='center', va='center')
        else:
            for i, coin in enumerate(features.index):
                ax3.annotate(coin, (features_pca[i, 0], features_pca[i, 1]),
                            fontsize=10, ha='center', va='center')

        if pca_success and n_components == 2:
            ax3.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)')
            ax3.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)')
        else:
            ax3.set_xlabel('Feature 1')
            ax3.set_ylabel('Feature 2')

        ax3.set_title('Cryptocurrency Clusters (PCA)')
        plt.colorbar(scatter, ax=ax3)
    else:
        ax3.text(0.5, 0.5, 'PCA visualization not available\n(Insufficient features)',
                ha='center', va='center', fontsize=12)
        ax3.set_title('Cryptocurrency Clusters')

    # 4. 특성별 히트맵
    ax4 = plt.subplot(3, 3, 4)
    features_with_cluster = features.copy()
    features_with_cluster['Cluster'] = clusters
    features_sorted = features_with_cluster.sort_values('Cluster')

    # 표시할 특성 선택 (존재하는 특성만)
    display_features = ['mean_return', 'volatility', 'sharpe_ratio', 'skewness',
                       'positive_periods_ratio', 'max_drawdown']
    display_features = [f for f in display_features if f in features.columns]

    # 히트맵 그리기
    if display_features and len(features_sorted) > 0:
        if len(features_sorted) > 50:
            sns.heatmap(features_sorted[display_features].T,
                        cmap='coolwarm', center=0,
                        xticklabels=False,
                        yticklabels=display_features,
                        ax=ax4, cbar_kws={'shrink': 0.8})
        else:
            sns.heatmap(features_sorted[display_features].T,
                        cmap='coolwarm', center=0,
                        xticklabels=features_sorted.index,
                        yticklabels=display_features,
                        ax=ax4, cbar_kws={'shrink': 0.8})
            plt.setp(ax4.xaxis.get_majorticklabels(), rotation=45, ha='right')
    else:
        ax4.text(0.5, 0.5, 'No features to display', ha='center', va='center')

    ax4.set_title('Feature Heatmap by Coin')

    # 5. 클러스터별 평균 특성
    ax5 = plt.subplot(3, 3, 5)
    if display_features:
        cluster_means = features_with_cluster.groupby('Cluster')[display_features].mean()

        # 안전한 정규화
        cluster_means_std = cluster_means.std()
        cluster_means_std[cluster_means_std == 0] = 1  # 0으로 나누기 방지
        cluster_means_normalized = (cluster_means - cluster_means.mean()) / cluster_means_std

        sns.heatmap(cluster_means_normalized.T, annot=True, fmt='.2f',
                    cmap='RdBu_r', center=0, ax=ax5,
                    xticklabels=[f'Cluster {i}' for i in sorted(cluster_df['Cluster'].unique())],
                    yticklabels=display_features,
                    cbar_kws={'shrink': 0.8})
    else:
        ax5.text(0.5, 0.5, 'No features to analyze', ha='center', va='center')

    ax5.set_title('Normalized Mean Features by Cluster')

    # 6. 수익률-변동성 산점도
    ax6 = plt.subplot(3, 3, 6)
    if 'volatility' in features.columns and 'mean_return' in features.columns:
        scatter2 = ax6.scatter(features['volatility']*100, features['mean_return']*100,
                              c=clusters, cmap='viridis', s=200, alpha=0.6)

        # 레이블 표시 (겹침 방지)
        if len(features.index) <= 20:
            for i, coin in enumerate(features.index):
                ax6.annotate(coin, (features['volatility'].iloc[i]*100,
                                   features['mean_return'].iloc[i]*100),
                            fontsize=9, ha='left', va='bottom')
        else:
            # 각 클러스터에서 대표 코인만 표시
            for cluster in sorted(cluster_df['Cluster'].unique()):
                cluster_coins = cluster_df[cluster_df['Cluster'] == cluster]['Coin'].values
                if len(cluster_coins) > 0:
                    cluster_features = features.loc[cluster_coins]
                    if len(cluster_features) > 0 and 'sharpe_ratio' in cluster_features.columns:
                        best_coin = cluster_features.nlargest(1, 'sharpe_ratio').index[0]
                        idx = features.index.get_loc(best_coin)
                        ax6.annotate(best_coin,
                                   (features['volatility'].iloc[idx]*100,
                                    features['mean_return'].iloc[idx]*100),
                                   fontsize=9, ha='left', va='bottom',
                                   bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7))

        ax6.set_xlabel('4H Volatility (%)')
        ax6.set_ylabel('Mean 4H Return (%)')
        ax6.grid(True, alpha=0.3)
        plt.colorbar(scatter2, ax=ax6)
    else:
        ax6.text(0.5, 0.5, 'Risk-return data not available', ha='center', va='center')

    ax6.set_title('Risk-Return Profile by Cluster')

    # 7. 클러스터별 박스플롯 (수익률)
    ax7 = plt.subplot(3, 3, 7)
    cluster_returns_list = []
    cluster_labels = []

    for cluster in sorted(cluster_df['Cluster'].unique()):
        coins_in_cluster = cluster_df[cluster_df['Cluster'] == cluster]['Coin'].values
        valid_coins = [coin for coin in coins_in_cluster if coin in returns_df.columns]

        if valid_coins:
            cluster_data = []
            for coin in valid_coins:
                coin_returns = returns_df[coin].dropna().values
                if len(coin_returns) > 0:
                    cluster_data.extend(coin_returns)

            if cluster_data:
                cluster_returns_list.append(cluster_data)
                cluster_labels.append(f'Cluster {cluster}')

    if cluster_returns_list:
        bp = ax7.boxplot(cluster_returns_list, labels=cluster_labels, patch_artist=True)
        colors = plt.cm.viridis(np.linspace(0, 1, len(cluster_returns_list)))
        for patch, color in zip(bp['boxes'], colors):
            patch.set_facecolor(color)
        ax7.set_ylabel('4H Returns')
        ax7.set_title('Returns Distribution by Cluster')
        ax7.grid(True, alpha=0.3)
    else:
        ax7.text(0.5, 0.5, 'No data available', ha='center', va='center')
        ax7.set_title('Returns Distribution by Cluster')

    # 8. 상관관계 히트맵 (클러스터별 정렬)
    ax8 = plt.subplot(3, 3, 8)
    valid_coins = [coin for coin in cluster_df['Coin'].values if coin in returns_df.columns]
    cluster_df_valid = cluster_df[cluster_df['Coin'].isin(valid_coins)]

    if len(cluster_df_valid) > 0:
        sorted_coins = cluster_df_valid.sort_values('Cluster')['Coin'].values
        corr_sorted = returns_df[sorted_coins].corr()

        # 상관관계 히트맵 그리기
        if len(sorted_coins) > 30:
            sns.heatmap(corr_sorted, cmap='coolwarm', center=0,
                        square=True, ax=ax8,
                        xticklabels=False, yticklabels=False,
                        cbar_kws={'shrink': 0.8})
        else:
            mask = np.zeros_like(corr_sorted, dtype=bool)
            mask[np.triu_indices_from(mask, k=1)] = True
            sns.heatmap(corr_sorted, mask=mask, cmap='coolwarm', center=0,
                        square=True, ax=ax8,
                        xticklabels=sorted_coins, yticklabels=sorted_coins,
                        cbar_kws={'shrink': 0.8})
            plt.setp(ax8.xaxis.get_majorticklabels(), rotation=45, ha='right')
            plt.setp(ax8.yaxis.get_majorticklabels(), rotation=0)

        ax8.set_title('Correlation Matrix (Sorted by Cluster)')
    else:
        ax8.text(0.5, 0.5, 'No data available', ha='center', va='center')
        ax8.set_title('Correlation Matrix')

    # 9. 클러스터 크기 파이 차트
    ax9 = plt.subplot(3, 3, 9)
    cluster_sizes = cluster_df['Cluster'].value_counts().sort_index()

    if len(cluster_sizes) > 0:
        colors = plt.cm.viridis(np.linspace(0, 1, len(cluster_sizes)))

        # 올바른 클러스터 번호와 크기 사용
        labels = [f'Cluster {cluster_num}\n({size} coins)'
                 for cluster_num, size in cluster_sizes.items()]

        wedges, texts, autotexts = ax9.pie(cluster_sizes.values,
                                            labels=labels,
                                            autopct='%1.1f%%',
                                            startangle=90,
                                            colors=colors)

        # 텍스트 스타일 조정
        for text in texts:
            text.set_fontsize(10)
        for autotext in autotexts:
            autotext.set_color('white')
            autotext.set_fontsize(9)
            autotext.set_weight('bold')
    else:
        ax9.text(0.5, 0.5, 'No clusters to display', ha='center', va='center')

    ax9.set_title('Cluster Size Distribution')

    plt.tight_layout()
    plt.show()
    plt.close(fig)  # 메모리 정리

    # 클러스터별 상세 정보 출력
    print("\n🔍 클러스터별 코인 그룹:")
    print("=" * 60)
    for cluster in sorted(cluster_df['Cluster'].unique()):
        coins_in_cluster = cluster_df[cluster_df['Cluster'] == cluster]['Coin'].values
        print(f"\n클러스터 {cluster} ({len(coins_in_cluster)}개 코인):")

        if len(coins_in_cluster) <= 20:
            print(f"  코인: {', '.join(coins_in_cluster)}")
        else:
            print(f"  코인: {', '.join(coins_in_cluster[:10])} ... 외 {len(coins_in_cluster)-10}개")

        # 클러스터 특성 요약
        if len(coins_in_cluster) > 0:
            valid_coins = [coin for coin in coins_in_cluster if coin in features.index]
            if valid_coins:
                cluster_features = features.loc[valid_coins].mean()
                print(f"  - 평균 4시간 수익률: {cluster_features['mean_return']*100:.4f}%")
                print(f"  - 평균 변동성: {cluster_features['volatility']*100:.4f}%")
                print(f"  - 평균 샤프 비율: {cluster_features['sharpe_ratio']:.4f}")
                print(f"  - 평균 양의 수익 구간 비율: {cluster_features['positive_periods_ratio']*100:.1f}%")
                if 'max_drawdown' in cluster_features and not np.isnan(cluster_features['max_drawdown']):
                    print(f"  - 평균 최대 낙폭: {cluster_features['max_drawdown']*100:.2f}%")
                if 'intraday_volatility' in cluster_features and not np.isnan(cluster_features['intraday_volatility']):
                    print(f"  - 평균 일중 변동성: {cluster_features['intraday_volatility']*100:.2f}%")

    # 특성 중요도
    if pca_success and n_components >= 2:
        print("\n📈 클러스터 구분에 중요한 특성:")
        print("=" * 60)
        feature_importance = np.abs(pca.components_).mean(axis=0)
        feature_names = list(features.columns)
        if len(feature_names) == len(feature_importance):
            importance_df = pd.DataFrame({
                'Feature': feature_names,
                'Importance': feature_importance
            }).sort_values('Importance', ascending=False)

            for idx, row in importance_df.head(8).iterrows():
                print(f"  {row['Feature']}: {row['Importance']:.3f}")

    return cluster_df, features

# 메인 실행 코드
if __name__ == "__main__":
    # 모든 KRW 마켓 코인 가져오기
    print("업비트 KRW 마켓 코인 목록을 가져오는 중...")
    all_krw_coins = get_all_krw_markets()

    if not all_krw_coins:
        print("❌ KRW 마켓 코인 목록을 가져올 수 없습니다.")
        exit()

    print(f"\n📊 업비트 KRW 마켓 분석")
    print("=" * 60)
    print(f"총 KRW 마켓 코인 수: {len(all_krw_coins)}개")

    # 분석 옵션 선택
    print("\n분석 옵션을 선택하세요:")
    print("1. 전체 코인 분석 (시간이 오래 걸림)")
    print("2. 상위 N개 코인만 분석")
    print("3. 주요 코인만 분석 (추천)")

    # 기본값: 전체 코인 분석
    choice = input("\n선택 (기본값: 3): ").strip() or "3"

    if choice == "1":
        coin_list = all_krw_coins
        print(f"\n전체 {len(coin_list)}개 코인을 분석합니다.")
    elif choice == "2":
        n = int(input("분석할 코인 수를 입력하세요 (예: 50): "))
        coin_list = all_krw_coins[:n]
        print(f"\n상위 {n}개 코인을 분석합니다.")
    else:
        # 주요 코인 목록 (거래량 상위 + 인기 코인)
        major_coins = ['BTC', 'ETH', 'XRP', 'ADA', 'SOL', 'DOGE', 'AVAX', 'MATIC',
                      'LINK', 'DOT', 'ATOM', 'UNI', 'BCH', 'LTC', 'ETC', 'ALGO',
                      'SAND', 'MANA', 'AXS', 'THETA', 'EOS', 'TRX', 'XLM', 'VET',
                      'HBAR', 'EGLD', 'NEAR', 'FLOW', 'CHZ', 'ENJ', 'QTUM', 'NEO']
        coin_list = [coin for coin in major_coins if coin in all_krw_coins]
        print(f"\n주요 {len(coin_list)}개 코인을 분석합니다.")

    # 과거 데이터 기간 설정 (4시간봉 기준)
    hours = 4320  # 180일 = 4320시간 = 6개월

    print(f"\n업비트에서 {hours}시간({hours//24}일 = {hours//24//30:.1f}개월)간의 4시간봉 데이터를 수집합니다...")
    print(f"예상 데이터 포인트: 약 {hours//4}개 (180일 × 6개/일)")

    # 데이터 수집 - 개선된 함수 사용
    returns_df, price_df, quality_groups = get_multiple_coins_data(coin_list, hours=hours)

    if returns_df is None or returns_df.empty:
        print("❌ 데이터 수집에 실패했습니다.")
        exit()

    print(f"\n✅ 분석 가능한 코인 수: {len(returns_df.columns)}개")
    print(f"수집된 데이터 기간: {returns_df.index[0].strftime('%Y-%m-%d %H:%M')} ~ {returns_df.index[-1].strftime('%Y-%m-%d %H:%M')}")
    print(f"총 4시간봉 개수: {len(returns_df)}개")

    # 클러스터링 분석 수행
    cluster_df, features = perform_clustering_analysis(returns_df)

    if cluster_df is None or features is None:
        print("❌ 클러스터링 분석에 실패했습니다.")
        exit()

    # 클러스터별 대표 코인 추천
    print("\n🎯 클러스터별 대표 코인 추천:")
    print("=" * 60)
    for cluster in sorted(cluster_df['Cluster'].unique()):
        cluster_coins = cluster_df[cluster_df['Cluster'] == cluster]['Coin'].values
        valid_coins = [coin for coin in cluster_coins if coin in features.index]

        if valid_coins:
            cluster_features = features.loc[valid_coins]

            # 샤프 비율이 높은 상위 3개 코인
            top_coins = cluster_features.nlargest(min(3, len(cluster_features)), 'sharpe_ratio')
            print(f"\n클러스터 {cluster} 추천 코인:")
            for coin in top_coins.index:
                sharpe = top_coins.loc[coin, 'sharpe_ratio']
                volatility = top_coins.loc[coin, 'volatility'] * 100
                mean_return = top_coins.loc[coin, 'mean_return'] * 100
                print(f"  - {coin}: 샤프비율 {sharpe:.3f}, 4시간 수익률 {mean_return:.3f}%, 변동성 {volatility:.3f}%")

    # 포트폴리오 구성 제안
    print("\n💡 포트폴리오 구성 제안:")
    print("=" * 60)
    print("1. 각 클러스터에서 샤프 비율이 높은 1-2개 코인 선택")
    print("2. 전체 포트폴리오의 5-10개 코인으로 구성")
    print("3. BTC와 ETH는 안정성을 위해 포함 권장")
    print("4. 각 코인의 비중은 리스크 허용도에 따라 조정")

    # 추가 분석: 포트폴리오 최적화 예시
    print("\n🎲 샘플 포트폴리오 구성:")
    print("=" * 60)
    sample_portfolio = []

    for cluster in sorted(cluster_df['Cluster'].unique()):
        cluster_coins = cluster_df[cluster_df['Cluster'] == cluster]['Coin'].values
        valid_coins = [coin for coin in cluster_coins if coin in features.index]

        if valid_coins:
            cluster_features = features.loc[valid_coins]
            # 각 클러스터에서 샤프비율 최고 코인 선택
            best_coin = cluster_features.nlargest(1, 'sharpe_ratio').index[0]
            sample_portfolio.append(best_coin)

    # BTC, ETH가 없으면 추가
    if 'BTC' in returns_df.columns and 'BTC' not in sample_portfolio:
        sample_portfolio.insert(0, 'BTC')
    if 'ETH' in returns_df.columns and 'ETH' not in sample_portfolio:
        sample_portfolio.insert(1, 'ETH')

    # 최대 10개로 제한
    sample_portfolio = sample_portfolio[:10]

    print(f"추천 포트폴리오 구성 ({len(sample_portfolio)}개 코인):")
    total_sharpe = 0
    for i, coin in enumerate(sample_portfolio):
        if coin in features.index:
            sharpe = features.loc[coin, 'sharpe_ratio']
            volatility = features.loc[coin, 'volatility'] * 100
            total_sharpe += sharpe
            print(f"  {i+1}. {coin} (샤프비율: {sharpe:.3f}, 변동성: {volatility:.2f}%)")

    if len(sample_portfolio) > 0:
        print(f"\n포트폴리오 평균 샤프비율: {total_sharpe/len(sample_portfolio):.3f}")

    # 분석 결과 저장 옵션
    save = input("\n분석 결과를 CSV로 저장하시겠습니까? (y/n): ").strip().lower()
    if save == 'y':
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

        try:
            # 클러스터 결과 저장
            cluster_df.to_csv(f'upbit_clustering_result_{timestamp}.csv',
                             index=False, encoding='utf-8-sig')

            # 특성 데이터 저장
            features.to_csv(f'upbit_coin_features_{timestamp}.csv',
                           encoding='utf-8-sig')

            # 추천 포트폴리오 저장
            portfolio_df = pd.DataFrame({
                'Rank': range(1, len(sample_portfolio)+1),
                'Coin': sample_portfolio,
                'Sharpe_Ratio': [features.loc[coin, 'sharpe_ratio'] if coin in features.index else np.nan
                               for coin in sample_portfolio],
                '4H_Return_%': [features.loc[coin, 'mean_return']*100 if coin in features.index else np.nan
                                 for coin in sample_portfolio],
                '4H_Volatility_%': [features.loc[coin, 'volatility']*100 if coin in features.index else np.nan
                                     for coin in sample_portfolio]
            })
            portfolio_df.to_csv(f'upbit_recommended_portfolio_{timestamp}.csv',
                              index=False, encoding='utf-8-sig')

            print(f"\n✅ 분석 결과가 저장되었습니다:")
            print(f"  - 클러스터 결과: upbit_clustering_result_{timestamp}.csv")
            print(f"  - 코인 특성: upbit_coin_features_{timestamp}.csv")
            print(f"  - 추천 포트폴리오: upbit_recommended_portfolio_{timestamp}.csv")

        except Exception as e:
            logger.error(f"파일 저장 중 오류 발생: {str(e)}")
            print(f"\n❌ 파일 저장 중 오류 발생: {str(e)}")

    print("\n📌 분석 완료!")

업비트 KRW 마켓 코인 목록을 가져오는 중...

📊 업비트 KRW 마켓 분석
총 KRW 마켓 코인 수: 179개

분석 옵션을 선택하세요:
1. 전체 코인 분석 (시간이 오래 걸림)
2. 상위 N개 코인만 분석
3. 주요 코인만 분석 (추천)


KeyboardInterrupt: Interrupted by user